In [ ]:
!git clone https://github.com/MinkyungPark/UsedCarPricePredict.git

fatal: destination path 'UsedCarPricePredict' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

In [4]:
dummy_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train-data.csv')
dummy_data.head()

FileNotFoundError: ignored

In [ ]:
# 데이터 구조 확인
print('row 수 : {}, col 수 : {}'.format(dummy_data.shape[0], dummy_data.shape[1]))
dummy_data.head()

In [ ]:
### 데이터 전처리

# NaN 결측치 제거
clean_data = dummy_data.copy(deep=True)
# 6019 -> 813 너무 많이 손실
clean_data.dropna('index').shape # (813,14)

# New_Price 컬럼의 이상치 확인
print(str(clean_data.New_Price.isna().sum())) # 5195개 na

# New_Price, Unnaed:0 컬럼 제거
clean_data.drop(columns=['Unnamed: 0', 'New_Price'], inplace=True)

clean_data.dropna("index", inplace = True)
clean_data = clean_data.reset_index(drop = True)
clean_data.shape # (5975, 12)

In [ ]:
# 차종류 수 정의

len(np.unique(list(clean_data.Name))) # 1855 가지
# 차 종별 갯수가 2개 이상인 종류만해도 1034종
# 모델을 너무 복잡하게 해서 overfitting 초래 할 수 있음

# 차종에서 브랜드만 남기고 제거
names = list(clean_data.Name)
for i in range(len(names)):
  names[i]=names[i].split(' ', 1)[0]
clean_data.Name = names
clean_data.head()

len(np.unique(list(clean_data.Name))) # 31 브랜드

In [ ]:
# Mileage의 km/kg제거 Engine의 CC제거, Power의 bhp제거

mileage = list(clean_data.Mileage)
engine = list(clean_data.Engine)
power = list(clean_data.Power)

for i in range(len(names)):
  mileage[i] = mileage[i].split(' ',1)[0]
  engine[i] = engine[i].split(' ',1)[0]
  power[i] = power[i].split(' ',1)[0]

clean_data.Mileage = mileage
clean_data.Engine = engine
clean_data.Power = power
clean_data.head()

In [ ]:
clean_data["Price"] = clean_data["Price"].astype(float)
clean_data["Kilometers_Driven"] = clean_data["Kilometers_Driven"].astype(float)
clean_data["Mileage"] = clean_data["Mileage"].astype(float)
clean_data["Engine"] = clean_data["Engine"].astype(float)
(clean_data.dtypes)

In [ ]:
clean_data.isnull().sum() # null, none확인
clean_data['Power'].isnull().sum()

In [ ]:
np.unique(list(clean_data.Name)) # 31

In [ ]:
np.unique(list(clean_data.Location))

In [ ]:
np.unique(list(clean_data.Year))

In [ ]:
np.unique(list(clean_data.Fuel_Type))

In [ ]:
np.unique(list(clean_data.Transmission))

In [ ]:
np.unique(list(clean_data.Owner_Type))

In [ ]:
np.unique(list(clean_data.Seats))
# array([ 0.,  2.,  4.,  5.,  6.,  7.,  8.,  9., 10.])   0 ....?
clean_data=clean_data[clean_data.Seats != 0] # Seats=0인것 제거

In [ ]:
print(clean_data.shape) # 5975->5974

In [ ]:
np.unique(list(clean_data.Power)) # null이 있으나 isnull.sum에서는 안나옴.. 왜지

# Power의 null 제거 후 float으로
idx = []
lt = list(clean_data['Power'])
for i in range(len(lt)):
  if( lt[i] == 'null'):
    idx.append(i)
clean_data = clean_data.drop(idx)
clean_data = clean_data.reset_index(drop = True)

In [ ]:
clean_data['Power'] = clean_data['Power'].astype(float)

In [ ]:
# 카테고리컬로 분류하기 Name, Location, Owner_Type, Seats

clean_data['Year'] = pd.Categorical(clean_data['Year'])
clean_data['Seats'] = pd.Categorical(clean_data['Seats'])

In [ ]:
clean_data = pd.get_dummies(clean_data, prefix_sep='_', drop_first=True)

In [ ]:
clean_data.head()

In [ ]:
print(clean_data.shape) # (5844, 79)

In [ ]:
### Outlier 확인

fig, ax = plt.subplots(1,5,figsize=(16,4))
ax[0].boxplot(list(clean_data.Kilometers_Driven))
ax[0].set_title("Kilometers Driven")

ax[1].boxplot(clean_data.Mileage)
ax[1].set_title("Mileage")

ax[2].boxplot(clean_data.Engine)
ax[2].set_title("Engine")
    
ax[3].boxplot(list(clean_data.Power))
ax[3].set_title("Power")
    
ax[4].boxplot(list(clean_data.Price))
ax[4].set_title("Price")
    
plt.show() # kilo가 6000000이상인 것 확인됨

In [ ]:
sns.pairplot(data=clean_data, x_vars=['Kilometers_Driven', 'Mileage', 'Engine', 'Power'], y_vars='Price', size=3)

In [ ]:
# kilo 6000000 삭제

idx = []
lt = list(clean_data["Kilometers_Driven"])
for i in range(len(lt)):   
    if( lt[i] > 1000000):
        idx.append(i)
clean_data = clean_data.drop(idx)
clean_data = clean_data.reset_index(drop = True)

In [ ]:
fig, ax = plt.subplots(1,5,figsize=(16,4))
ax[0].boxplot(list(clean_data.Kilometers_Driven))
ax[0].set_title("Kilometers Driven")

ax[1].boxplot(clean_data.Mileage)
ax[1].set_title("Mileage")

ax[2].boxplot(clean_data.Engine)
ax[2].set_title("Engine")
    
ax[3].boxplot(list(clean_data.Power))
ax[3].set_title("Power")
    
ax[4].boxplot(list(clean_data.Price))
ax[4].set_title("Price")
    
plt.show()

In [ ]:
sns.pairplot(data=clean_data, x_vars=['Kilometers_Driven', 'Mileage', 'Engine', 'Power'], y_vars='Price', size=3)

In [ ]:
### train 데이터 생성

y = clean_data[['Price']].to_numpy()
clean_data = clean_data.drop(columns=['Price'])

In [ ]:
x = clean_data.values
columns = clean_data.columns

# Scale (x값만)
scaler = preprocessing.MinMaxScaler()
tmp = scaler.fit_transform(x)
clean_data = pd.DataFrame(tmp)
clean_data.columns = columns

x = clean_data.to_numpy()

In [ ]:
# data split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.85, random_state=1)

In [ ]:
### 모델링 -> 다중 선형 회귀 사용 y = w1*x1 + w2*x2 +....+ b

lr = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
lr.fit(x_train, y_train)

In [ ]:
# train data 정확도 R2(R-squared) 측정
print('Train dats\'s Accuracy : ', format(lr.score(x_train, y_train)))

In [ ]:
y_predict = lr.predict(x_test)

In [ ]:
# test dats 정확도 R2(R-squared) 측정
print('Test dats\'s Accuracy : ', format(lr.score(x_test,y_test)))
print('Test dats\'s Accuracy : ', format(r2_score(y_test,lr.predict(x_test))))

Train acc와 Test acc의 값이 거의 유사하다. overfitting은 일어나지 않았으나 test data 사이즈를 늘이거나 random state를 바꾸면 test acc가 급격히 떨어진다..

In [ ]:
mean_absolute_error(y_test, y_predict)

Price의 오차 296000루피 한화 480 만원..